In [1]:
def get_index_lst(send_date, timepoints_lst, cond_ind = "<"):
    
    if type(timepoints_lst) == type(list()):
        if (cond_ind == "<") | (cond_ind.lower() == "prior"):
            return [x[0] for x in enumerate(timepoints_lst) if x[1] < send_date]
        elif (cond_ind == "=") | (cond_ind.lower() == "present"):
            return [x[0] for x in enumerate(timepoints_lst) if x[1] == send_date]
        elif (cond_ind == ">") | (cond_ind.lower() == "posterior"):
            return [x[0] for x in enumerate(timepoints_lst) if x[1] > send_date]
        else:
            return print("Error: Condition Indicator should be one of '<', '=', or '>'.")
    else:
        return None


In [1]:
import pandas as pd

fold_pth = "~/personal_ds7900/"

email_2020_hist_pth = fold_pth + "temporal_subsets/email_history_subsets/" + "ALL_2020" + ".csv"
full_email_hist_pth = fold_pth + "original_data/" + "email_hist" + ".csv"
stay_hist_pth = fold_pth + "original_data/" + "stay_hist" + ".csv"
members_pth = fold_pth + "original_data/" + "members" + ".csv"

email_hist_coltype = {"HASH_NBR":"str", "CAMPAIGN_NBR":"str", "CLICK":"str", "SEND_DT":"str", "MBR_TIER":"str", "MBR_PRGM_ACTV":"str", "CAMPAIGN_NM":"str", "UNSUB_IND":"str"}
stay_hist_coltype = {"HASH_NBR":"str", "CONF_HASH_NBR":"str", "CONF_DT":"str", "CK_IN_DT":"str", "CK_OUT_DT":"str", "HTL_HASH_NBR":"str", "HTL_RGN":"str", "HTL_CTRY_NM":"str", "HTL_CITY_NM":"str", "HTL_CHAIN":"str", "GUEST_QTY":"str", "REWARD_NT":"str", "NBR_OF_NIGHTS":"str", "ROOM_REVENUE_USD":"str", "BUS_LEIS_IND":"str", "HTL_CHAIN_CATEGORY":"str"}
members_coltype = {"HASH_NBR":"str", "ENROLL_DT":"str", "ENROLL_CHANNEL":"str", "MBR_REGION":"str", "MBR_SUBREGION":"str", "STATE_NM":"str", "CITY_NM":"str", "AGE_CD":"str", "INCOME_CD":"str", "GENDER_CD":"str", "INCOME_GROUP":"str"}


In [3]:

email_hist_pth = data_fold_pth + "temporal_subsets/email_history_subsets/" + "Jan_2021" + ".csv"


###################################################################################################
### Import Data & Merge Stay Data with Email Data
###################################################################################################

email_hist_df = pd.merge(
    left = pd.read_csv(email_hist_pth, dtype = email_hist_coltype, nrows = 1),
    right = pd.read_csv(stay_hist_pth, dtype = stay_hist_coltype).groupby("HASH_NBR").agg(list).reset_index(),
    left_on = "HASH_NBR",
    right_on = "HASH_NBR",
    how = "left"
)

###################################################################################################
### Separate Prior, Present, and Posterior Stay Data relative to Email Send Date
###################################################################################################

for position_nm in [("prior", "<"), ("present", "="), ("posterior", ">")]:

    email_hist_df["stay_index_list_" + position_nm[0]] = [get_index_lst(send_date, ck_out_lst, cond_ind = position_nm[1]) for send_date, ck_out_lst in zip(email_hist_df["SEND_DT_timepoint"], email_hist_df["CK_OUT_timepoint"])]
    
    for col_nm in pd.read_csv(stay_hist_pth, dtype = stay_hist_coltype, nrows = 1).drop("HASH_NBR", axis = 1).columns:
        email_hist_df[position_nm[0] + "_" + col_nm] = [[x[1] for x in enumerate(pull_from) if x[0] in criteria_list] if type(pull_from) == type(list()) else None for pull_from, criteria_list in zip(email_hist_df[col_nm], email_hist_df["stay_index_list_" + position_nm[0]])]


###################################################################################################
### Merge Email Data with its Member-Aggregated Form
###################################################################################################

email_hist_df = pd.merge(
    left = email_hist_df,
    right = pd.read_csv(email_hist_pth, dtype = email_hist_coltype).groupby("HASH_NBR").agg(list).reset_index(),
    left_on = "HASH_NBR",
    right_on = "HASH_NBR",
    how = "left",
    suffixes = ("", "_list")
)

###################################################################################################
### Separate Prior Member-Aggregated Email Data relative to Email Send Date
###################################################################################################

email_hist_df["email_index_list_prior"] = [get_index_lst(send_date, ck_out_lst, cond_ind = "<") for send_date, ck_out_lst in zip(email_hist_df["SEND_DT_timepoint"], email_hist_df["SEND_DT_timepoint_list"])]
email_hist_df["email_index_list_present"] = [get_index_lst(send_date, ck_out_lst, cond_ind = "=") for send_date, ck_out_lst in zip(email_hist_df["SEND_DT_timepoint"], email_hist_df["SEND_DT_timepoint_list"])]
for col_nm in [x + "_list" for x in pd.read_csv(email_hist_pth, dtype = email_hist_coltype, nrows = 1).drop("HASH_NBR", axis = 1).columns]:
    email_hist_df["prior_" + col_nm] = [[x[1] for x in enumerate(pull_from) if x[0] in criteria_list] if type(pull_from) == type(list()) else None for pull_from, criteria_list in zip(email_hist_df[col_nm], email_hist_df["email_index_list_prior"])]

###################################################################################################
### Merge 2020 Data with Prior Email Data
###################################################################################################



###################################################################################################
### 
###################################################################################################

email_hist_df.head(3)


,HASH_NBR,CAMPAIGN_NBR,UNSUB_IND,SEND_DT,CLICK,CAMPAIGN_NM,MBR_TIER,MBR_PRGM_ACTV,yr,mnth,...,prior_CLICK_list,prior_CAMPAIGN_NM_list,prior_MBR_TIER_list,prior_MBR_PRGM_ACTV_list,prior_yr_list,prior_mnth_list,prior_SEND_DT_timepoint_list,prior_centered_SEND_DT_timepoint_list,prior_gr_ind_list,prior_dy_list
0,8896147939408129278,-8323003949268062675,0,2021-01-19,0,CC_1,TIER_1,LFC_3,2021,Jan,...,"[0, 0, 0]","[CC_4, CC_2, CC_1]","[TIER_1, TIER_1, TIER_1]","[LFC_3, LFC_3, LFC_3]","[2021, 2021, 2021]","[Jan, Jan, Jan]","[2021.12440109514, 2021.099760438056, 2021.118...","[1.038329911019673, 1.0136892539355813, 1.0328...","[1, 1, 1]","[15, 6, 13]"


In [4]:
list(email_hist_df.columns)

['HASH_NBR',
 'CAMPAIGN_NBR',
 'UNSUB_IND',
 'SEND_DT',
 'CLICK',
 'CAMPAIGN_NM',
 'MBR_TIER',
 'MBR_PRGM_ACTV',
 'yr',
 'mnth',
 'SEND_DT_timepoint',
 'centered_SEND_DT_timepoint',
 'gr_ind',
 'dy',
 'CONF_HASH_NBR',
 'CONF_DT',
 'CK_IN_DT',
 'CK_OUT_DT',
 'HTL_HASH_NBR',
 'HTL_RGN',
 'HTL_CTRY_NM',
 'HTL_CITY_NM',
 'HTL_CHAIN',
 'GUEST_QTY',
 'REWARD_NT',
 'NBR_OF_NIGHTS',
 'ROOM_REVENUE_USD',
 'BUS_LEIS_IND',
 'HTL_CHAIN_CATEGORY',
 'CONF_timepoint',
 'centered_CONF_timepoint',
 'CK_IN_timepoint',
 'centered_CK_IN_timepoint',
 'CK_OUT_timepoint',
 'centered_CK_OUT_timepoint',
 'CK_INOUT_timepoint_diff',
 'stay_index_list_prior',
 'prior_CONF_HASH_NBR',
 'prior_CONF_DT',
 'prior_CK_IN_DT',
 'prior_CK_OUT_DT',
 'prior_HTL_HASH_NBR',
 'prior_HTL_RGN',
 'prior_HTL_CTRY_NM',
 'prior_HTL_CITY_NM',
 'prior_HTL_CHAIN',
 'prior_GUEST_QTY',
 'prior_REWARD_NT',
 'prior_NBR_OF_NIGHTS',
 'prior_ROOM_REVENUE_USD',
 'prior_BUS_LEIS_IND',
 'prior_HTL_CHAIN_CATEGORY',
 'prior_CONF_timepoint',
 'prio

In [ ]:
email_hist_df

In [ ]:
'CONF_HASH_NBR',

'CONF_DT',
'CK_IN_DT',
'CK_OUT_DT',

'HTL_HASH_NBR',
'HTL_RGN', 
'HTL_CTRY_NM', 
'HTL_CITY_NM',
'HTL_CHAIN',
'HTL_CHAIN_CATEGORY',

'GUEST_QTY',
'REWARD_NT',
'NBR_OF_NIGHTS',
'ROOM_REVENUE_USD',
'BUS_LEIS_IND',

'CONF_timepoint',
'CK_IN_timepoint',
'CK_INOUT_timepoint_diff', 

'centered_CONF_timepoint',
'centered_CK_IN_timepoint',
'centered_CK_OUT_timepoint',